## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
v = model.encode(user_question)
v[0]

0.07822262

## Q2. Creating the embeddings

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
documents=[document for document in documents if document['course']=="machine-learning-zoomcamp"]

In [7]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [8]:
import numpy as np
from tqdm import tqdm
embeddings=[]
for doc in tqdm(documents):
    qa_text = f'{doc["question"]} {doc["text"]}'
    embeddings.append(model.encode(qa_text))
X = np.array(embeddings)    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:50<00:00,  7.38it/s]


In [9]:
X.shape

(375, 768)

## Q3. Search

In [10]:
scores = X.dot(v)
scores.max()

0.6506575

## Q4. Hit-rate for our search engine

In [11]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [12]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [13]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [14]:
from tqdm.auto import tqdm 

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    question = q['question']
    v_question = model.encode(question)
    results = search_engine.search(v_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

cnt = 0

for line in relevance_total:
    if True in line:
        cnt = cnt + 1

  0%|          | 0/1830 [00:00<?, ?it/s]

In [15]:
hitrate = cnt / len(relevance_total)
hitrate

0.9398907103825137

## Q5. Indexing with Elasticsearch

In [16]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
for i, embedding in enumerate(embeddings):
    documents[i]['question_text_vector'] = embedding

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [19]:
knn = {
    "field": 'question_text_vector',
    "query_vector": v,
    "k": 5,
    "num_candidates": 10000
}

search_query = {
    "knn": knn,
    "_source": ["text", "section", "question", "course", "id"]
}

es_results = es_client.search(
    index=index_name,
    body=search_query
)

es_results['hits']['hits']

[{'_index': 'course-questions',
  '_id': '14JoupAB-JwuFfGwtAVP',
  '_score': 0.8253288,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'}},
 {'_index': 'course-questions',
  '_id': '2oJoupAB-JwuFfGwtAW_',
  '_score': 0.7358538,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Welcome to the course! Go to the 

In [28]:
es_results['hits']['hits'][0]['_source']['id']

'c6a22665'

## Q6. Hit-rate for Elasticsearch

In [20]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    question = q['question']
    v_question = model.encode(question)

    knn = {
    "field": 'question_text_vector',
    "query_vector": v_question,
    "k": 5,
    "num_candidates": 10000
    }
    search_query = {
    "knn": knn,
    "_source": ["text", "section", "question", "course", "id"]
    }
    
    es_results = es_client.search(
    index=index_name,
    body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])


    relevance = [d['id'] == doc_id for d in result_docs]
    relevance_total.append(relevance)

cnt = 0

for line in relevance_total:
    if True in line:
        cnt = cnt + 1

hitrate = cnt / len(relevance_total)
hitrate

  0%|          | 0/1830 [00:00<?, ?it/s]

0.9398907103825137